In [1]:
import json
from pathlib import Path
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape
from shapely.ops import unary_union
import pandas as pd

In [2]:
CRZ_GEOJSON_PATH = Path(
    r"NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson"
)

Method_1

In [3]:
def safe_int_from_val(val):
    if val is None:
        return None

    if isinstance(val, list):
        if not val:
            return None
        return safe_int_from_val(val[0])

    if isinstance(val, (int, float)):
        try:
            v = float(val)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    if isinstance(val, str):
        s = val.split(";")[0].strip()
        if s == "":
            return None
        try:
            v = float(s)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    return None


def get_lanes_from_tags(data):

    lanes = data.get("lanes", None)
    lanes_int = safe_int_from_val(lanes)
    if lanes_int is not None:
        return lanes_int

    lf = data.get("lanes:forward", None)
    lb = data.get("lanes:backward", None)

    lf_int = safe_int_from_val(lf)
    lb_int = safe_int_from_val(lb)

    if lf_int is not None or lb_int is not None:
        lf_val = lf_int if lf_int is not None else 0
        lb_val = lb_int if lb_int is not None else 0
        total = lf_val + lb_val
        if total > 0:
            return total

    return None


def main():
    if not CRZ_GEOJSON_PATH.exists():
        raise FileNotFoundError(f"GeoJSON not found: {CRZ_GEOJSON_PATH}")

    print(f"Loading CRZ boundary from: {CRZ_GEOJSON_PATH}")
    with open(CRZ_GEOJSON_PATH, "r", encoding="utf-8") as f:
        crz_geojson = json.load(f)

    polys = [shape(feat["geometry"]) for feat in crz_geojson["features"]]
    crz_polygon = unary_union(polys)

    print("Downloading OSM road network within CRZ polygon (network_type='drive') ...")
    G = ox.graph_from_polygon(crz_polygon, network_type="drive", simplify=True)
    Gu = G.to_undirected()

    total_length_m = 0.0
    total_lane_m = 0.0

    for u, v, data in Gu.edges(data=True):
        length = data.get("length", 0.0) or 0.0
        lanes_val = get_lanes_from_tags(data)
        if lanes_val is None:
            ##################
            lanes_used = 3.02 
        else:
            lanes_used = lanes_val

        total_length_m += length
        total_lane_m += length * lanes_used

    total_length_km = total_length_m / 1000.0
    total_lane_km = total_lane_m / 1000.0

    print("\n===== Lane-km Estimate within CRZ (missing lanes filled by mean) =====")
    print(f"Total centerline road length in CRZ: {total_length_km:.2f} km")
    print(f"Total lane-km of roads in CRZ:       {total_lane_km:.2f} lane-km")

if __name__ == "__main__":
    main()

Loading CRZ boundary from: NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson

===== Lane-km Estimate within CRZ (missing lanes filled by mean) =====
Total centerline road length in CRZ: 11131.79 km
Total lane-km of roads in CRZ:       30121.36 lane-km


Method 2

In [4]:
def safe_int_from_val(val):
    if val is None:
        return None

    if isinstance(val, list):
        if not val:
            return None
        return safe_int_from_val(val[0])

    if isinstance(val, (int, float)):
        try:
            v = float(val)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    if isinstance(val, str):
        s = val.split(";")[0].strip()
        if s == "":
            return None
        try:
            v = float(s)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    return None


def get_lanes_from_tags(data):

    lanes = data.get("lanes", None)
    lanes_int = safe_int_from_val(lanes)
    if lanes_int is not None:
        return lanes_int

    lf = data.get("lanes:forward", None)
    lb = data.get("lanes:backward", None)

    lf_int = safe_int_from_val(lf)
    lb_int = safe_int_from_val(lb)

    if lf_int is not None or lb_int is not None:
        lf_val = lf_int if lf_int is not None else 0
        lb_val = lb_int if lb_int is not None else 0
        total = lf_val + lb_val
        if total > 0:
            return total

    return None


def main():
    if not CRZ_GEOJSON_PATH.exists():
        raise FileNotFoundError(f"GeoJSON not found: {CRZ_GEOJSON_PATH}")

    print(f"Loading CRZ boundary from: {CRZ_GEOJSON_PATH}")
    with open(CRZ_GEOJSON_PATH, "r", encoding="utf-8") as f:
        crz_geojson = json.load(f)

    polys = [shape(feat["geometry"]) for feat in crz_geojson["features"]]
    crz_polygon = unary_union(polys)

    print("Downloading OSM road network within CRZ polygon (network_type='drive') ...")
    G = ox.graph_from_polygon(crz_polygon, network_type="drive", simplify=True)
    Gu = G.to_undirected()

    total_length_m = 0.0
    total_lane_m = 0.0

    for u, v, data in Gu.edges(data=True):
        length = data.get("length", 0.0) or 0.0
        lanes_val = get_lanes_from_tags(data)
        if lanes_val is None:
            ##################
            lanes_used = 3.510087679

        else:
            lanes_used = lanes_val

        total_length_m += length
        total_lane_m += length * lanes_used

    total_length_km = total_length_m / 1000.0
    total_lane_km = total_lane_m / 1000.0

    print("\n===== Lane-km Estimate within CRZ (missing lanes filled by mean) =====")
    print(f"Total centerline road length in CRZ: {total_length_km:.2f} km")
    print(f"Total lane-km of roads in CRZ:       {total_lane_km:.2f} lane-km")

if __name__ == "__main__":
    main()

Loading CRZ boundary from: NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson

===== Lane-km Estimate within CRZ (missing lanes filled by mean) =====
Total centerline road length in CRZ: 11131.79 km
Total lane-km of roads in CRZ:       33411.98 lane-km


Method 3

In [5]:
def safe_int_from_val(val):
    if val is None:
        return None

    if isinstance(val, list):
        if not val:
            return None
        return safe_int_from_val(val[0])

    if isinstance(val, (int, float)):
        try:
            v = float(val)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    if isinstance(val, str):
        s = val.split(";")[0].strip()
        if s == "":
            return None
        try:
            v = float(s)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    return None


def get_lanes_from_tags(data):

    lanes = data.get("lanes", None)
    lanes_int = safe_int_from_val(lanes)
    if lanes_int is not None:
        return lanes_int

    lf = data.get("lanes:forward", None)
    lb = data.get("lanes:backward", None)

    lf_int = safe_int_from_val(lf)
    lb_int = safe_int_from_val(lb)

    if lf_int is not None or lb_int is not None:
        lf_val = lf_int if lf_int is not None else 0
        lb_val = lb_int if lb_int is not None else 0
        total = lf_val + lb_val
        if total > 0:
            return total

    return None


def main():
    if not CRZ_GEOJSON_PATH.exists():
        raise FileNotFoundError(f"GeoJSON not found: {CRZ_GEOJSON_PATH}")

    print(f"Loading CRZ boundary from: {CRZ_GEOJSON_PATH}")
    with open(CRZ_GEOJSON_PATH, "r", encoding="utf-8") as f:
        crz_geojson = json.load(f)

    polys = [shape(feat["geometry"]) for feat in crz_geojson["features"]]
    crz_polygon = unary_union(polys)

    print("Downloading OSM road network within CRZ polygon (network_type='drive') ...")
    G = ox.graph_from_polygon(crz_polygon, network_type="drive", simplify=True)
    Gu = G.to_undirected()

    total_length_m = 0.0
    total_lane_m = 0.0

    for u, v, data in Gu.edges(data=True):
        length = data.get("length", 0.0) or 0.0
        lanes_val = get_lanes_from_tags(data)
        if lanes_val is None:
            ##################
            lanes_used = 2.137644652

        else:
            lanes_used = lanes_val

        total_length_m += length
        total_lane_m += length * lanes_used

    total_length_km = total_length_m / 1000.0
    total_lane_km = total_lane_m / 1000.0

    print("\n===== Lane-km Estimate within CRZ (missing lanes filled by mean) =====")
    print(f"Total centerline road length in CRZ: {total_length_km:.2f} km")
    print(f"Total lane-km of roads in CRZ:       {total_lane_km:.2f} lane-km")

if __name__ == "__main__":
    main()

Loading CRZ boundary from: NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson

===== Lane-km Estimate within CRZ (missing lanes filled by mean) =====
Total centerline road length in CRZ: 11131.79 km
Total lane-km of roads in CRZ:       24196.93 lane-km


Method 4

In [6]:
s = 1
MEAN_LANES_BY_HIGHWAY = {
    "residential":     1.500687893 * s,
    "secondary":       2.523068948 * s,
    "primary":         3.040925574 * s,
    "tertiary":        1.90782241 * s,
    "motorway_link":   1.297082228 * s,
    "motorway":        2.899382171 * s,
    "unclassified":    1.805706522 * s,
    "trunk":           3.247139588 * s,
    "primary_link":    1.184549356 * s,
    "secondary_link":  1.184210526 * s,
    "tertiary_link":   1.319148936 * s,
    "trunk_link":      1.555555556 * s,
    "living_street":   1 * s,
    "busway":          1 * s,
}


DEFAULT_LANES_IF_UNKNOWN = 1.0  


def load_crz_polygon(geojson_path: Path):
    if not geojson_path.exists():
        raise FileNotFoundError(f"GeoJSON not found: {geojson_path}")
    print(f"Loading CRZ boundary from: {geojson_path}")
    with open(geojson_path, "r", encoding="utf-8") as f:
        crz_geojson = json.load(f)
    polys = [shape(feat["geometry"]) for feat in crz_geojson["features"]]
    crz_polygon = unary_union(polys)
    return crz_polygon


def normalize_highway(val):

    if isinstance(val, list):
        if not val:
            return None
        return val[0]
    return val


def safe_int_from_val(val):

    if val is None:
        return None

    if isinstance(val, list):
        if not val:
            return None
        return safe_int_from_val(val[0])

    if isinstance(val, (int, float)):
        try:
            v = float(val)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    if isinstance(val, str):
        s = val.split(";")[0].strip()
        if s == "":
            return None
        try:
            v = float(s)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    return None


def get_lanes_from_tags(row):

    lanes = row.get("lanes", None)
    lanes_int = safe_int_from_val(lanes)
    if lanes_int is not None:
        return lanes_int

    lf = row.get("lanes:forward", None)
    lb = row.get("lanes:backward", None)
    lf_int = safe_int_from_val(lf)
    lb_int = safe_int_from_val(lb)

    if lf_int is not None or lb_int is not None:
        lf_val = lf_int if lf_int is not None else 0
        lb_val = lb_int if lb_int is not None else 0
        total = lf_val + lb_val
        if total > 0:
            return total

    return None


def lane_filled_for_row(row):
    lanes_val = get_lanes_from_tags(row)
    if lanes_val is not None:
        return float(lanes_val)

    h = row.get("highway_norm", None)
    if h in MEAN_LANES_BY_HIGHWAY:
        return float(MEAN_LANES_BY_HIGHWAY[h])

    return float(DEFAULT_LANES_IF_UNKNOWN)


def main():
    crz_polygon = load_crz_polygon(CRZ_GEOJSON_PATH)

    print("Downloading OSM road network within CRZ polygon (network_type='drive') ...")
    G = ox.graph_from_polygon(crz_polygon, network_type="drive", simplify=True)
    Gu = G.to_undirected()

    edges = ox.graph_to_gdfs(Gu, nodes=False, edges=True)

    edges["highway_norm"] = edges["highway"].apply(normalize_highway)

    for col in ["lanes", "lanes:forward", "lanes:backward"]:
        if col not in edges.columns:
            edges[col] = pd.NA

    edges["has_lane_info"] = (
        edges["lanes"].notna()
        | edges["lanes:forward"].notna()
        | edges["lanes:backward"].notna()
    )

    print("Filling missing lane counts by type-specific mean...")
    edges["lanes_filled"] = edges.apply(lane_filled_for_row, axis=1)

    edges["length_m"] = edges["length"].fillna(0.0)

    total_length_m = edges["length_m"].sum()
    total_lane_m = (edges["length_m"] * edges["lanes_filled"]).sum()

    total_length_km = total_length_m / 1000.0
    total_lane_km = total_lane_m / 1000.0

    total_edges = len(edges)
    missing_edges = (~edges["has_lane_info"]).sum()

    print("\n===== CRZ results with type-specific mean lane imputation =====")
    print(f"Total edges in CRZ network:        {total_edges}")
    print(f"Edges with NO original lane tags:  {missing_edges}")
    print(f"Total centerline length in CRZ:    {total_length_km:.2f} km")
    print(f"Total lane-km in CRZ (imputed):    {total_lane_km:.2f} lane-km")


if __name__ == "__main__":
    main()

Loading CRZ boundary from: NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson
Filling missing lane counts by type-specific mean...

===== CRZ results with type-specific mean lane imputation =====
Total edges in CRZ network:        91765
Edges with NO original lane tags:  58988
Total centerline length in CRZ:    11131.79 km
Total lane-km in CRZ (imputed):    20582.24 lane-km


Method 5

In [7]:
s = 1.854935184
MEAN_LANES_BY_HIGHWAY = {
    "residential":     1.500687893 * s,
    "secondary":       2.523068948 * s,
    "primary":         3.040925574 * s,
    "tertiary":        1.90782241 * s,
    "motorway_link":   1.297082228 * s,
    "motorway":        2.899382171 * s,
    "unclassified":    1.805706522 * s,
    "trunk":           3.247139588 * s,
    "primary_link":    1.184549356 * s,
    "secondary_link":  1.184210526 * s,
    "tertiary_link":   1.319148936 * s,
    "trunk_link":      1.555555556 * s,
    "living_street":   1 * s,
    "busway":          1 * s,
}


DEFAULT_LANES_IF_UNKNOWN = 1.0  


def load_crz_polygon(geojson_path: Path):
    if not geojson_path.exists():
        raise FileNotFoundError(f"GeoJSON not found: {geojson_path}")
    print(f"Loading CRZ boundary from: {geojson_path}")
    with open(geojson_path, "r", encoding="utf-8") as f:
        crz_geojson = json.load(f)
    polys = [shape(feat["geometry"]) for feat in crz_geojson["features"]]
    crz_polygon = unary_union(polys)
    return crz_polygon


def normalize_highway(val):

    if isinstance(val, list):
        if not val:
            return None
        return val[0]
    return val


def safe_int_from_val(val):

    if val is None:
        return None

    if isinstance(val, list):
        if not val:
            return None
        return safe_int_from_val(val[0])

    if isinstance(val, (int, float)):
        try:
            v = float(val)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    if isinstance(val, str):
        s = val.split(";")[0].strip()
        if s == "":
            return None
        try:
            v = float(s)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    return None


def get_lanes_from_tags(row):

    lanes = row.get("lanes", None)
    lanes_int = safe_int_from_val(lanes)
    if lanes_int is not None:
        return lanes_int

    lf = row.get("lanes:forward", None)
    lb = row.get("lanes:backward", None)
    lf_int = safe_int_from_val(lf)
    lb_int = safe_int_from_val(lb)

    if lf_int is not None or lb_int is not None:
        lf_val = lf_int if lf_int is not None else 0
        lb_val = lb_int if lb_int is not None else 0
        total = lf_val + lb_val
        if total > 0:
            return total

    return None


def lane_filled_for_row(row):
    lanes_val = get_lanes_from_tags(row)
    if lanes_val is not None:
        return float(lanes_val)

    h = row.get("highway_norm", None)
    if h in MEAN_LANES_BY_HIGHWAY:
        return float(MEAN_LANES_BY_HIGHWAY[h])

    return float(DEFAULT_LANES_IF_UNKNOWN)


def main():
    crz_polygon = load_crz_polygon(CRZ_GEOJSON_PATH)

    print("Downloading OSM road network within CRZ polygon (network_type='drive') ...")
    G = ox.graph_from_polygon(crz_polygon, network_type="drive", simplify=True)
    Gu = G.to_undirected()

    edges = ox.graph_to_gdfs(Gu, nodes=False, edges=True)

    edges["highway_norm"] = edges["highway"].apply(normalize_highway)

    for col in ["lanes", "lanes:forward", "lanes:backward"]:
        if col not in edges.columns:
            edges[col] = pd.NA

    edges["has_lane_info"] = (
        edges["lanes"].notna()
        | edges["lanes:forward"].notna()
        | edges["lanes:backward"].notna()
    )

    print("Filling missing lane counts by type-specific mean...")
    edges["lanes_filled"] = edges.apply(lane_filled_for_row, axis=1)

    edges["length_m"] = edges["length"].fillna(0.0)

    total_length_m = edges["length_m"].sum()
    total_lane_m = (edges["length_m"] * edges["lanes_filled"]).sum()

    total_length_km = total_length_m / 1000.0
    total_lane_km = total_lane_m / 1000.0

    total_edges = len(edges)
    missing_edges = (~edges["has_lane_info"]).sum()

    print("\n===== CRZ results with type-specific mean lane imputation =====")
    print(f"Total edges in CRZ network:        {total_edges}")
    print(f"Edges with NO original lane tags:  {missing_edges}")
    print(f"Total centerline length in CRZ:    {total_length_km:.2f} km")
    print(f"Total lane-km in CRZ (imputed):    {total_lane_km:.2f} lane-km")


if __name__ == "__main__":
    main()

Loading CRZ boundary from: NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson
Filling missing lane counts by type-specific mean...

===== CRZ results with type-specific mean lane imputation =====
Total edges in CRZ network:        91765
Edges with NO original lane tags:  58988
Total centerline length in CRZ:    11131.79 km
Total lane-km in CRZ (imputed):    29762.71 lane-km


Method 6

In [8]:
s = 2.155955342
MEAN_LANES_BY_HIGHWAY = {
    "residential":     1.500687893 * s,
    "secondary":       2.523068948 * s,
    "primary":         3.040925574 * s,
    "tertiary":        1.90782241 * s,
    "motorway_link":   1.297082228 * s,
    "motorway":        2.899382171 * s,
    "unclassified":    1.805706522 * s,
    "trunk":           3.247139588 * s,
    "primary_link":    1.184549356 * s,
    "secondary_link":  1.184210526 * s,
    "tertiary_link":   1.319148936 * s,
    "trunk_link":      1.555555556 * s,
    "living_street":   1 * s,
    "busway":          1 * s,
}


DEFAULT_LANES_IF_UNKNOWN = 1.0  


def load_crz_polygon(geojson_path: Path):
    if not geojson_path.exists():
        raise FileNotFoundError(f"GeoJSON not found: {geojson_path}")
    print(f"Loading CRZ boundary from: {geojson_path}")
    with open(geojson_path, "r", encoding="utf-8") as f:
        crz_geojson = json.load(f)
    polys = [shape(feat["geometry"]) for feat in crz_geojson["features"]]
    crz_polygon = unary_union(polys)
    return crz_polygon


def normalize_highway(val):

    if isinstance(val, list):
        if not val:
            return None
        return val[0]
    return val


def safe_int_from_val(val):

    if val is None:
        return None

    if isinstance(val, list):
        if not val:
            return None
        return safe_int_from_val(val[0])

    if isinstance(val, (int, float)):
        try:
            v = float(val)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    if isinstance(val, str):
        s = val.split(";")[0].strip()
        if s == "":
            return None
        try:
            v = float(s)
            if v <= 0:
                return None
            return int(round(v))
        except Exception:
            return None

    return None


def get_lanes_from_tags(row):

    lanes = row.get("lanes", None)
    lanes_int = safe_int_from_val(lanes)
    if lanes_int is not None:
        return lanes_int

    lf = row.get("lanes:forward", None)
    lb = row.get("lanes:backward", None)
    lf_int = safe_int_from_val(lf)
    lb_int = safe_int_from_val(lb)

    if lf_int is not None or lb_int is not None:
        lf_val = lf_int if lf_int is not None else 0
        lb_val = lb_int if lb_int is not None else 0
        total = lf_val + lb_val
        if total > 0:
            return total

    return None


def lane_filled_for_row(row):
    lanes_val = get_lanes_from_tags(row)
    if lanes_val is not None:
        return float(lanes_val)

    h = row.get("highway_norm", None)
    if h in MEAN_LANES_BY_HIGHWAY:
        return float(MEAN_LANES_BY_HIGHWAY[h])

    return float(DEFAULT_LANES_IF_UNKNOWN)


def main():
    crz_polygon = load_crz_polygon(CRZ_GEOJSON_PATH)

    print("Downloading OSM road network within CRZ polygon (network_type='drive') ...")
    G = ox.graph_from_polygon(crz_polygon, network_type="drive", simplify=True)
    Gu = G.to_undirected()

    edges = ox.graph_to_gdfs(Gu, nodes=False, edges=True)

    edges["highway_norm"] = edges["highway"].apply(normalize_highway)

    for col in ["lanes", "lanes:forward", "lanes:backward"]:
        if col not in edges.columns:
            edges[col] = pd.NA

    edges["has_lane_info"] = (
        edges["lanes"].notna()
        | edges["lanes:forward"].notna()
        | edges["lanes:backward"].notna()
    )

    print("Filling missing lane counts by type-specific mean...")
    edges["lanes_filled"] = edges.apply(lane_filled_for_row, axis=1)

    edges["length_m"] = edges["length"].fillna(0.0)

    total_length_m = edges["length_m"].sum()
    total_lane_m = (edges["length_m"] * edges["lanes_filled"]).sum()

    total_length_km = total_length_m / 1000.0
    total_lane_km = total_lane_m / 1000.0

    total_edges = len(edges)
    missing_edges = (~edges["has_lane_info"]).sum()

    print("\n===== CRZ results with type-specific mean lane imputation =====")
    print(f"Total edges in CRZ network:        {total_edges}")
    print(f"Edges with NO original lane tags:  {missing_edges}")
    print(f"Total centerline length in CRZ:    {total_length_km:.2f} km")
    print(f"Total lane-km in CRZ (imputed):    {total_lane_km:.2f} lane-km")


if __name__ == "__main__":
    main()

Loading CRZ boundary from: NYC_Neighborhood_Tabulation_Areas_2020_-2131974656277759428.geojson
Filling missing lane counts by type-specific mean...

===== CRZ results with type-specific mean lane imputation =====
Total edges in CRZ network:        91765
Edges with NO original lane tags:  58988
Total centerline length in CRZ:    11131.79 km
Total lane-km in CRZ (imputed):    32995.12 lane-km
